In [26]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_wine())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])

In [27]:
print(x.shape,y.shape) # (178, 13) (178, 1)
print(y[:5])

# y의 클래스 값 분류 + 불균형확인
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]


(178, 13) (178, 1)
   0
0  0
1  0
2  0
3  0
4  0
[0 1 2]
(array([0, 1, 2]), array([59, 71, 48], dtype=int64))


In [28]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )
# print(x.columns)
# y 의 첫번째 칼럼을 원핫 인코딩 변환
# y_train[0] = tf.one_hot(y_train[0],3)
# y_test[0] = tf.one_hot(y_test[0],3)
print(y_train[:5])
print("shape : ",y_train.shape)

     0
28   0
99   1
62   1
164  2
117  1
shape :  (142, 1)


In [29]:
model = Sequential([
    Dense(50,activation="relu",input_shape=(13,)),
    Dense(40,activation="relu") ,
    Dense(40,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(20,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [30]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="sparse_categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
4/4 [==============================] - 0s 48ms/step - loss: 14.7344 - accuracy: 0.2832 - val_loss: 9.6182 - val_accuracy: 0.3448
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 10.1327 - accuracy: 0.3274 - val_loss: 7.6967 - val_accuracy: 0.3448
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 6.9529 - accuracy: 0.3894 - val_loss: 5.4791 - val_accuracy: 0.3103
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 5.1182 - accuracy: 0.2478 - val_loss: 3.7611 - val_accuracy: 0.3448
Epoch 5/100
4/4 [==============================] - 0s 10ms/step - loss: 3.5058 - accuracy: 0.3274 - val_loss: 2.6704 - val_accuracy: 0.4138
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 2.6548 - accuracy: 0.3894 - val_loss: 2.1146 - val_accuracy: 0.3793
Epoch 7/100
4/4 [==============================] - 0s 10ms/step - loss: 1.6745 - accuracy: 0.3717 - val_loss: 0.8489 - val_accuracy: 0.5862
Epoch 8/100
4/4 [=

In [31]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


2/2 [==============================] - 0s 4ms/step - loss: 0.5525 - accuracy: 0.6944
loss: 0.5525143146514893 
acc : 0.6944444179534912


In [33]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print(y_predict[:5])
# y_test 를 원핫 인코딩 해제 해야함
print(y_test[:5])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)

[[0.05466958 0.5216007  0.42372966]
 [0.02895259 0.8273524  0.14369504]
 [0.02493191 0.7282793  0.2467887 ]
 [0.41090173 0.28698963 0.3021086 ]
 [0.22239135 0.40059936 0.37700933]]
[1 1 1 0 1]
     0
165  0
105  0
85   0
82   0
129  0
0.3888888888888889
